In [1]:
import numpy as np
import plotly.graph_objects as go
import requests
import json
import datetime
import csv
import codecs
import urllib
import matplotlib.pyplot as plt
import pandas as pd
import time
import glob

In [9]:
def load_data(fname):
    data = pd.read_csv(fname)
    # data.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
    # cleanup whitespaces in data
    # data= data.applymap(lambda x: x.strip() if type(x)==str else x)
    #. Add columns in which the time on the dongle is converted UNIX EPOCH time, and "datetime64"
    #  convert to MDT by subtracting 6*3600
    # data['datetime'] = (data['local_time']-6*3600).astype('datetime64[s]')
    return data

In [3]:
files = glob.glob('*los.csv')
files.sort()

In [23]:
def plotly_rssi(t,rssi):
    #. Adapted from https://github.com/plotly/plotly.py/issues/1445
    fig = go.FigureWidget()
        
    fig.add_scattergl(x=t, y=rssi, 
                      mode='markers', 
                      marker = dict(size=5, symbol='cross', color='blue')
                     )
    fig.add_histogram(y=rssi, name='rssi', marker=dict(opacity=0.7, color='blue'), 
                      xaxis='x2'
                     )
    fig.data[1].meta = rssi

    fig.layout = dict(xaxis=dict(domain=[0, 0.84], showgrid=False, zeroline=False),
                      yaxis=dict(domain=[0, 1], showgrid=False, zeroline=False),
                      showlegend=True,
                      margin=dict(t=50),
                      hovermode='closest',
                      bargap=0.25,
                      xaxis2=dict(domain=[0.85, 1], showgrid=False, zeroline=False),
                      height=600,
                      title={'text':f, 'x':0.42, 'xanchor':'center'}
                     )
    #  Lots of trouble getting the updating the histogram to work... Seems to work now that
    #.  I have wrapped the whole thing in a function... Works on local jupyter
    #.  Not working colab jupyter
    # 
    def do_zoom(layout, xaxis_range, yaxis_range):
        #global y
        inds = ((xaxis_range[0] <= t) & (t <= xaxis_range[1]) &
                (yaxis_range[0] <= rssi) & (rssi <= yaxis_range[1]))

        with fig.batch_update():
            fig.data[1].y = fig.data[1].meta[inds]


    fig.layout.on_change(do_zoom, 'xaxis.range', 'yaxis.range')

    return fig

In [25]:
fig={}
for f in files:
    print(f)
    data = load_data(f);
    fig[f] = plotly_rssi(data['time'], data['RSSI'])
    fig[f].show()

0.50_los.csv


0.75_los.csv


1.00_los.csv


1.25_los.csv


1.50_los.csv


...

2.25_los.csv


2.50_los.csv


2.75_los.csv


3.00_los.csv


3.25_los.csv


In [85]:
files = glob.glob('*los.csv')
files.sort()
hist2d=[];
for f in files:
    #print(f)
    data = load_data(f);
    column = np.zeros(100*2)
    hist = np.array(np.unique(data['RSSI'], return_counts=True)).T
    for row in hist:
        row_idx = -int(row[0]/0.5)
        column[row_idx] = row[1]/len(data['RSSI'])
    hist2d.append(column)
hist2d = np.array(hist2d).T
fig = go.Figure(data=go.Heatmap(
                    x=np.arange(0.5, 3.5, 0.25),
                    y=np.arange(-20, -100, -0.5),
                    z=hist2d[40:90,:]))
fig.update_layout(
    title={
        'text': "900 MHz, Line of Sight data",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Separation [m]",
    yaxis_title="Normalized histogram",
    font=dict(
        size=18,
    )
)
fig.show()

In [89]:
files = glob.glob('*blocked.csv')
files.sort()
hist2d=[];
for f in files:
    #print(f)
    data = load_data(f);
    column = np.zeros(2*45)
    hist = np.array(np.unique(data['RSSI'], return_counts=True)).T
    for row in hist:
        row_idx = -int(row[0]/0.5)
        column[row_idx] = row[1]/len(data['RSSI'])
    hist2d.append(column)
hist2d = np.array(hist2d).T

fig = go.Figure(data=go.Heatmap(
                    x=np.arange(0.5, 3.5, 0.25),
                    y=np.arange(-20, -100, -0.5),
                    z=hist2d[40:90,:]))
fig.update_layout(
    title={
        'text': "900 MHz, Jug of water (fake human) block xmitter",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Separation [m]",
    yaxis_title="Normalized histogram",
    font=dict(
        size=18,
    )
)
fig.show()